In [ ]:
import pandas as pd

In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
import random
class Node:
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

moves = {(-1,0): 'up', (0,-1): 'left', (1,0): 'down', (0,1): 'right'}
moves_int = {'left': 0, 'down': 1, 'right': 2, 'up': 3}
moves_to_coord = {'up': (-1,0), 'left': (0,-1), 'down': (1,0), 'right': (0,1)}

def astar(maze, start, end):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                if current.parent is not None:
                  move = tuple(a-b for a, b in zip(current.position, current.parent.position))
                  path.append(moves[move])
                current = current.parent
            return path[::-1]  # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:  # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze) - 1]) - 1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] == '1':
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)



def generate_random_position(grid_size):
    """
    Generate a random position within a grid of the given size.

    Args:
    grid_size (int): The size of the grid (grid_size x grid_size).

    Returns:
    tuple: A tuple representing a random position (row, column) in the grid.
    """
    row = random.randint(0, grid_size - 1)
    column = random.randint(0, grid_size - 1)
    return row, column


In [ ]:
def run(num_iterations):
  df = pd.DataFrame(columns=['Prompt', 'Answer'])
  for _ in range(num_iterations):
      S_position_i , S_position_j = generate_random_position(4)
      G_position_i, G_position_j = generate_random_position(4)

      while True:
        if S_position_i == G_position_i and S_position_j == G_position_j:
          G_position_i, G_position_j = generate_random_position(4)
        else:
          break

      # Define the maze
      maze = [['F', 'F', 'F', 'F'],
              ['F', 'F', 'F', 'F'],
              ['F', 'F', 'F', 'F'],
              ['F', 'F', 'F', 'F'],
              ]

      maze[S_position_i][S_position_j] = 'S'
      maze[G_position_i][G_position_j] = 'G'

      # print(maze)
      # Define start and end
      start = None
      end = None
      for i in range(len(maze)):
          for j in range(len(maze[i])):
              if maze[i][j] == 'S':
                  start = (i, j)
              elif maze[i][j] == 'G':
                  end = (i, j)

      if start and end:
          path = astar(maze, start, end)
          # path = [moves_int[move] for move in path]
          # print("Path:", path)
          # print("Start or goal not defined in the maze")
      else:
        continue

      S = (S_position_i , S_position_j)
      G = (G_position_i, G_position_j)

      # Generating the movements based on the path
      samples = []
      current_pos = S
      for step in path:
          x, y = current_pos
          sample = []
          sample.append(f'Your current position: ({x},{y}), you have to reach ({G[0]},{G[1]}). Answer : {step}')

          dx, dy = moves_to_coord[step]
          current_pos = (x+dx,y+dy)
          samples.append(sample)

      df_run = pd.DataFrame(samples, columns=['Prompt'])
      df = pd.concat([df, df_run], ignore_index=True)
  return df

In [ ]:
df = run(10)
df

,Prompt,Answer
0,"Your current position: (2,3), you have to reac...",NaN
1,"Your current position: (2,2), you have to reac...",NaN
2,"Your current position: (2,1), you have to reac...",NaN
3,"Your current position: (1,3), you have to reac...",NaN
4,"Your current position: (1,2), you have to reac...",NaN
5,"Your current position: (1,1), you have to reac...",NaN
6,"Your current position: (1,0), you have to reac...",NaN
7,"Your current position: (2,1), you have to reac...",NaN
8,"Your current position: (2,2), you have to reac...",NaN
9,"Your current position: (2,3), you have to reac...",NaN


In [ ]:
df.to_csv('Data_A_star.csv')

In [ ]:
df.Prompt

0     Your current position: (2,3), you have to reac...
1     Your current position: (2,2), you have to reac...
2     Your current position: (2,1), you have to reac...
3     Your current position: (1,3), you have to reac...
4     Your current position: (1,2), you have to reac...
5     Your current position: (1,1), you have to reac...
6     Your current position: (1,0), you have to reac...
7     Your current position: (2,1), you have to reac...
8     Your current position: (2,2), you have to reac...
9     Your current position: (2,3), you have to reac...
10    Your current position: (1,2), you have to reac...
11    Your current position: (1,1), you have to reac...
12    Your current position: (2,3), you have to reac...
13    Your current position: (2,2), you have to reac...
14    Your current position: (2,1), you have to reac...
15    Your current position: (2,0), you have to reac...
16    Your current position: (0,3), you have to reac...
17    Your current position: (1,3), you have to 

In [ ]:
'''User:
You are a navigator bot provide context of how I want to control navigation. you output directions, Top, Bottom, Right and Left.

Look at the following data and its Answers:
Your current position: (2,3), you have to reach (1,1). Answer : left
Your current position: (2,2), you have to reach (1,1). Answer : left
Your current position: (2,1), you have to reach (1,1). Answer : up
Your current position: (1,3), you have to reach (2,0). Answer : left
Your current position: (1,2), you have to reach (2,0). Answer : left
Your current position: (1,1), you have to reach (2,0). Answer : left
Your current position: (1,0), you have to reach (2,0). Answer : down
Your current position: (2,1), you have to reach (3,3). Answer : right
Your current position: (2,2), you have to reach (3,3). Answer : right
Your current position: (2,3), you have to reach (3,3). Answer : down
Your current position: (1,2), you have to reach (1,0). Answer : left
Your current position: (1,1), you have to reach (1,0). Answer : left
Your current position: (2,3), you have to reach (1,0). Answer : left
Your current position: (2,2), you have to reach (1,0). Answer : left
Your current position: (2,1), you have to reach (1,0). Answer : left
Your current position: (2,0), you have to reach (1,0). Answer : up
Your current position: (0,3), you have to reach (3,1). Answer : down
Your current position: (1,3), you have to reach (3,1). Answer : left
Your current position: (1,2), you have to reach (3,1). Answer : down
Your current position: (2,2), you have to reach (3,1). Answer : left
Your current position: (2,1), you have to reach (3,1). Answer : down
Your current position: (3,3), you have to reach (2,3). Answer : up
Your current position: (3,1), you have to reach (2,3). Answer : right
Your current position: (3,2), you have to reach (2,3). Answer : right
Your current position: (3,3), you have to reach (2,3). Answer : up
Your current position: (2,2), you have to reach (2,0). Answer : left
Your current position: (2,1), you have to reach (2,0). Answer : left
Your current position: (0,2), you have to reach (2,1). Answer : down
Your current position: (1,2), you have to reach (2,1). Answer : left
Your current position: (1,1), you have to reach (2,1). Answer : down

Now Answer:
Your current position: (2,2), you have to reach (4,4). Answer which direction you move in. Options A) Up B) Down C) Left D) Right.'''

"User:\nYou are a navigator bot provide context of how I want to control navigation. you output directions, Top, Bottom, Right and Left.\nLook at the following current positions and following steps:\nYour current position: (0,3), you have to reach (3,1). Answer which direction you move in. Options A) Up B) Down C) Left D) Right.\n\nAssistant: \nLets think step by step:\n'Step 1: Current Position: (0,3), Target: (3,1). Analyze: Need to move down to reach row 3 and then left to reach column 1. Next Move: Down.\n Step 2: Current Position: (1,3), Target: (3,1). Analyze: Still need to move down to row 3, but also left to column 1. Next Move: Left.\n Step 3: Current Position: (1,2), Target: (3,1). Analyze: Continue moving down to row 3, then left to column 1. Next Move: Down. \nStep 4: Current Position: (2,2), Target: (3,1). Analyze: Move down to row 3, followed by left to column 1. Next Move: Left. \nStep 5: Current Position: (2,1), Target: (3,1). Analyze: Only need to move down to align wi